In [15]:
import keypress_recognition.models as models
from keypress_recognition import dataset_p as dataset
from keypress_recognition import dataset_t
import numpy as np
import matplotlib.pyplot as plt
import cv2
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
concat = False
delta = 3 if concat else 1
channels = 3 * delta

In [3]:
mod = {}
mod['black'] = models.get_black_key_model(channels)
mod['black'].load_model('keypress_recognition/models/best_black_model.tar')
mod['white'] = models.get_white_key_model(channels)
mod['white'].load_model('keypress_recognition/models/best_white_model.tar') 

Trying to load model from keypress_recognition/models/best_black_model.tar ...
done
Trying to load model from keypress_recognition/models/best_white_model.tar ...
done


In [4]:
dataset.load_all_data(color=['black', 'white'], size=['single'], keypress=True, concatenate=concat, delta=delta)

Start seperating keyboard ...
  White single width: 21px
  Black single width: 20px
  White bundle width: 37px
  Black bundle width: 36px


IntProgress(value=0, max=14273)

In trainset: 
  # of pressed black key: 8755
  # of unpressed black key: 43822
  # of pressed white key: 19083
  # of unpressed white key: 57344


IntProgress(value=0, max=5000)

In testset: 
  # of pressed black key: 761
  # of unpressed black key: 22609
  # of pressed white key: 10831
  # of unpressed white key: 23167


IntProgress(value=0, max=2993)

In valset: 
  # of pressed black key: 817
  # of unpressed black key: 10821
  # of pressed white key: 5090
  # of unpressed white key: 12125


In [5]:
def visualize_keys(note):
    """
    note: bool
    return: str
    """
    return ("Y" if note else "N")

def display_concat_img(img, idx=None):
    if concat:
        if idx is None:
            return img[:, :, :, 3:6]
        else:
            return img[idx, :, :, 3:6]
    if idx is None:
        return img
    else:
        return img[idx, :, :, :]


In [10]:
for color in ('black', 'white'):
    results = mod[color].test(dataset, size='single', color=color, concatenate=concat)
    for k, v in results.items():
        print("{:10}: {:.4f}".format(k.title(), v))

IntProgress(value=0, max=44980)

Precision : 0.9820
Recall    : 0.9921
Accuracy  : 0.9870
F1 Score  : 0.9870


IntProgress(value=0, max=45996)

Precision : 0.9263
Recall    : 0.9495
Accuracy  : 0.9370
F1 Score  : 0.9378


In [32]:
npy = np.empty((0, 88), dtype=bool)
for whites, blacks in dataset_t.data_batch(size='single'):
#     print(blacks.shape, whites.shape)
    blackout = mod['black'].evaluate(blacks).numpy()
    whiteout = mod['white'].evaluate(whites).numpy()
#     print(blackout.shape, whiteout.shape)
#     print(blackout.dtype, whiteout.dtype)
    out = dataset_t.merge_two_colors(whiteout, blackout, dtype=bool)
    npy = np.concatenate((npy, out[None]), axis=0)
#     np.save(f'{color}_prediction.npy', pred[color])

IntProgress(value=0, max=5000)

In [42]:
print(npy.shape)
np.save('keypress_recognition/dataset/y_pred/y_pred_single.npy', npy)
# print(blackout)
# print(whiteout)
# print(npy)
# print(npy[0, np.array([27, 29, 31, 34])])

(5000, 88)


In [43]:
# sanity check
label = np.load('keypress_recognition/dataset/y_test/y_test.npy')
label = label[:, 21:109] > 0
print(label.shape)
accuracy = np.mean(label == npy)
print(accuracy)

(5000, 88)
0.9158568181818182
